#1. Install requirements

In [11]:
!pip install -q fastembed
!pip install -q qdrant-client
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#2. Dataset

In [3]:
!gdown --fuzzy https://drive.google.com/file/d/17x7am_JGRXfiwNO68mv3yOCpbLK-7IDA/view?usp=drive_link


Downloading...
From (original): https://drive.google.com/uc?id=17x7am_JGRXfiwNO68mv3yOCpbLK-7IDA
From (redirected): https://drive.google.com/uc?id=17x7am_JGRXfiwNO68mv3yOCpbLK-7IDA&confirm=t&uuid=003772c4-4736-4697-a40b-e3663ae9424d
To: /content/train.csv
100% 185M/185M [00:01<00:00, 124MB/s]


In [4]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,question,context,cid,qid
0,Người học ngành quản lý khai thác công trình t...,"['Khả năng học tập, nâng cao trình độ\n- Khối ...",[62492],161615
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,['Nội dung lồng ghép vấn đề bình đẳng giới tro...,[151154],80037
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"['""Điều 20. Ưu đãi về thời gian miễn thuế, giả...",[75071],124074
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"['Điều kiện được hưởng\nCán bộ quản lý, giáo v...",[225897],146841
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"['Nguyên tắc áp dụng\n1. Trường hợp công chức,...",[68365],6176


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119456 entries, 0 to 119455
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  119456 non-null  object
 1   context   119456 non-null  object
 2   cid       119456 non-null  object
 3   qid       119456 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 3.6+ MB


In [6]:
print("question: ", df['question'][0])
print("qid: ", df['qid'][0])
print()
print("context: ", df['context'][0])
print("cid: ", df['cid'][0])

question:  Người học ngành quản lý khai thác công trình thủy lợi trình độ cao đẳng phải có khả năng học tập và nâng cao trình độ như thế nào?
qid:  161615

context:  ['Khả năng học tập, nâng cao trình độ\n- Khối lượng kiến thức tối thiểu, yêu cầu về năng lực mà người học phải đạt được sau khi tốt nghiệp ngành, nghề quản lý, khai thác các công trình thủy lợi, trình độ cao đẳng có thể tiếp tục phát triển ở các trình độ cao hơn;\n- Người học sau tốt nghiệp có năng lực tự học, tự cập nhật những tiến bộ khoa học công nghệ trong phạm vi ngành, nghề để nâng cao trình độ hoặc học liên thông lên trình độ cao hơn trong cùng ngành, nghề hoặc trong nhóm ngành, nghề hoặc trong cùng lĩnh vực đào tạo./.']
cid:  [62492]


In [7]:
import pandas as pd

from tqdm import tqdm
from pathlib import Path


def preprocess_row_data(row: pd.Series) -> list[dict]:
    """
    Preprocess row data from BKAI
    """
    result: list[dict] = []
    question = row["question"].replace('"', "").replace("'", "").strip("\n").strip()
    qid = row["qid"]

    cid = [int(x) for x in row["cid"].strip("[] ").split(" ") if x]
    context = row["context"].strip("[]").split("'\n")

    assert len(cid) == len(context), f"{len(cid)} != {len(context)} for {row}"
    for i in range(len(cid)):
        result.append(
            {
                "question": question,
                "context": context[i]
                .replace('"', "")
                .replace("'", "")
                .strip("\n")
                .strip(),
                "cid": cid[i],
                "qid": qid,
            }
        )

    return result


df = pd.read_csv("train.csv")
columns = df.columns

all_rows = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    all_rows.extend(preprocess_row_data(row))

processed_df = pd.DataFrame(all_rows)

processed_df.to_csv("preprocessed_train.csv", index=False)


100%|██████████| 119456/119456 [00:10<00:00, 11433.27it/s]


In [8]:
df_processed = pd.read_csv("preprocessed_train.csv")
df_processed.head()

,question,context,cid,qid
0,Người học ngành quản lý khai thác công trình t...,"Khả năng học tập, nâng cao trình độ\n- Khối lư...",62492,161615
1,Nội dung lồng ghép vấn đề bình đẳng giới trong...,Nội dung lồng ghép vấn đề bình đẳng giới trong...,151154,80037
2,Sản phẩm phần mềm có được hưởng ưu đãi về thời...,"Điều 20. Ưu đãi về thời gian miễn thuế, giảm t...",75071,124074
3,Điều kiện để giáo viên trong cơ sở giáo dục mầ...,"Điều kiện được hưởng\nCán bộ quản lý, giáo viê...",225897,146841
4,Nguyên tắc áp dụng phụ cấp ưu đãi nghề y tế th...,"Nguyên tắc áp dụng\n1. Trường hợp công chức, v...",68365,6176


In [9]:
df_processed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 133568 entries, 0 to 133567
Data columns (total 4 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   question  133568 non-null  object
 1   context   133568 non-null  object
 2   cid       133568 non-null  int64 
 3   qid       133568 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 4.1+ MB


In [10]:
print("question: ", df_processed['question'][52])
print("qid: ", df_processed['qid'][52])
print()
print("context: ", df_processed['context'][52])
print("cid: ", df_processed['cid'][52])

question:  Hành vi làm tiền giả sẽ bị xử lý như thế nào?
qid:  83167

context:  “1. Người nào làm, tàng trữ, vận chuyển, lưu hành tiền giả, thì bị phạt tù từ 03 năm đến 07 năm.\n2. Phạm tội trong trường hợp tiền giả có trị giá tương ứng từ 5.000.000 đồng đến dưới 50.000.000 đồng, thì bị phạt tù từ 05 năm đến 12 năm.\n3. Phạm tội trong trường hợp tiền giả có trị giá tương ứng từ 50.000.000 đồng trở lên, thì bị phạt tù từ 10 năm đến 20 năm hoặc tù chung thân.\n4. Người chuẩn bị phạm tội này, thì bị phạt cải tạo không giam giữ đến 03 năm hoặc phạt tù từ 01 năm đến 03 năm.\n5. Người phạm tội còn có thể bị phạt tiền từ 10.000.000 đồng đến 100.000.000 đồng hoặc tịch thu một phần hoặc toàn bộ tài sản.”
cid:  65268


# 3. Ensemble retriever

In [ ]:
# https://qdrant.tech/articles/vector-search-filtering/

import uuid
import os
import json
import logging
import pandas as pd
from datasets import Dataset
from tqdm import tqdm
from fastembed import SparseTextEmbedding, TextEmbedding
from fastembed.late_interaction import LateInteractionTextEmbedding
from qdrant_client import QdrantClient
from qdrant_client import models
from qdrant_client.models import Filter
from google.colab import userdata


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class QdrantEngine:
    def __init__ (self):
        self.client = QdrantClient(
            url=userdata.get('QDRANT_CLOUD_ID'),
            api_key=userdata.get("QDRANT_API_KEY"),
        )
        self.index_name = "hackathon"
        self.jina_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-base-en")
        self.bm42_model = SparseTextEmbedding(model_name="Qdrant/bm25")
        self.late_model = LateInteractionTextEmbedding(model_name="colbert-ir/colbertv2.0")
        self.dataframe = pd.read_excel("preprocessed_train.xlsx")

    def _count_data(self):
        return self.client.count(collection_name=self.index_name)

    def _delete_colection(self):
        self.client.delete_collection(collection_name=self.index_name)

    def create_collection(self):
        self.client.recreate_collection(
            collection_name=self.index_name,
            optimizers_config=models.OptimizersConfigDiff(indexing_threshold=10000),
            hnsw_config=models.HnswConfigDiff(
                m=32,  # Increase the number of edges per node from the default 16 to 32
                ef_construct=200,  # Increase the number of neighbours from the default 100 to 200
            ),
            quantization_config=models.ScalarQuantization(
                scalar=models.ScalarQuantizationConfig(
                    type=models.ScalarType.INT8,
                    quantile=0.99,
                    always_ram=True
                )
            ),
            vectors_config={
                "jina-embeddings-v2": models.VectorParams(
                    size=768,
                    distance=models.Distance.COSINE,
                ),
                "colbertv2.0": models.VectorParams(
                    size=128,
                    distance=models.Distance.COSINE,
                    multivector_config=models.MultiVectorConfig(
                        comparator=models.MultiVectorComparator.MAX_SIM,
                    )
                ),
            },
            sparse_vectors_config={
                "bm25": models.SparseVectorParams(
                    modifier=models.Modifier.IDF,
                )
            }
        )

        logger.info(self.client.get_collection(collection_name=self.index_name))

    def update_data(self):

        batch_size = 4
        dataset = Dataset.from_pandas(self.dataframe, preserve_index=False)
        for batch in tqdm(dataset.iter(batch_size=batch_size), total=len(dataset) // batch_size):

            dense_embedding = list(self.jina_model.embed(documents=batch['context']))
            bm42_embedding = list(self.bm42_model.embed(documents=batch['context']))
            late_embedding = list(self.late_model.embed(documents=batch['context']))

            self.client.upload_points(
                collection_name=self.index_name,
                points= [
                    models.PointStruct(
                        id=uuid.uuid4().hex,

                        payload={
                            'cid': batch['cid'][i],
                            'qid': batch['qid'][i],
                        },
                        vector={
                            "jina-embeddings-v2": dense_embedding[i].tolist(),
                            "bm25": bm42_embedding[i].as_object(),
                            "colbertv2.0": late_embedding[i].tolist()
                        },
                    ) for i, _ in enumerate(batch["group_name"])
                ],
                batch_size=batch_size
            )

        logger.info(f"Uploading data to Qdrant successful !!")

    def search(self,
               query: str,
               pair_filter: dict[str, any] = None):

        sparse_embedding = list(self.bm42_model.embed(documents=query))[0]
        dense_embedding = list(self.jina_model.embed(documents=query))[0]
        late_embedding = list(self.late_model.embed(documents=query))[0]

        search_result = self.client.query_points(
            collection_name=self.index_name,
            prefetch=[
                models.Prefetch(query=sparse_embedding.as_object(), using="bm25", limit=5),
                models.Prefetch(query=dense_embedding.tolist(), using="jina-embeddings-v2", limit=5),
                models.Prefetch(query=late_embedding.tolist(), using="colbertv2.0", limit=5),
            ],
            query=models.FusionQuery(fusion=models.Fusion.RRF), # <--- Combine the scores of the two embeddings
            score_threshold=None,
            with_payload=True,
            with_vectors=False,
            limit=5,
        ).points
        return search_result

    def format_output(self, output_qdrant: list):
        outtext = ""
        for index, point in enumerate(output_qdrant):
            point = json.loads(json.dumps(point.dict(), indent=4))
            payload = point['payload']
            outtext +=  f"""
                {index+1}. "cid": {payload['cid']},
                "qid": {payload['qid']},
                "context": {payload['context']} \n"""
        return outtext

    def testing(self, query: str):
        self.create_collection()
        # self.update_data()

        results = self.search(query)
        print(self.format_output(results))

if __name__ == "__main__":
    query = "bán cho tôi Bộ Lưu Trữ Năng Lượng Mặt Trời SUNTEK 12V/25Ah PLUS"
    engine = QdrantEngine()
    engine.testing(query=query)

100%|██████████| 133568/133568 [00:15<00:00, 8633.07it/s] 
<ipython-input-14-c59c6d148dc1>:52: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  return Chroma(


tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

ValidationError: 1 validation error for ContextualCompressionRetriever
base_compressor
  value is not a valid dict (type=type_error.dict)